In [ ]:
import re
import numpy as np
import pandas as pd
import json
import glob

In [ ]:
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 50)

In [ ]:
input_filespecs = [
    "../../../mnt/lab9-isilon/shared/eaglemonk/examples-data-project-pvc-*/test-logs/*_longevity-test-*.log",
#     "../../../tmp/test-longevity-camera-claudio-04.log",
]

In [ ]:
records = []
for input_filespec in input_filespecs:
    for input_filename in sorted(glob.glob(input_filespec)):
        print(input_filename)
        with open(input_filename) as file:
            for line in file:
                m = re.match('{.*', line)
                if m:
                    rec1 = json.loads(line)
                    rec = rec1['fields'] 
                    rec['level'] = rec1['level']
                    rec['target'] = rec1['target']
                    records += [rec]

In [ ]:
df = pd.DataFrame(records)
if "pts" not in df:
    df["pts"] = np.nan
if "prev_pts" not in df:
    df["prev_pts"] = np.nan
if "time_delta" not in df:
    df["time_delta"] = np.nan
df["min_pts"] = pd.to_datetime(df["min_pts"], errors="coerce")
df["max_pts"] = pd.to_datetime(df["max_pts"], errors="coerce")
df["pts"] = pd.to_datetime(df["pts"], errors="coerce")
df["prev_pts"] = pd.to_datetime(df["prev_pts"], errors="coerce")
df["ms_from_prev"] = df.pts.diff() / pd.Timedelta(1, "millisecond")

In [ ]:
df.info()

In [ ]:
df.head(4)

In [ ]:
stats = df[df.description.isin(["statistics"])].groupby(["stream", "probe_name"]).last()

In [ ]:
pd.Timestamp.utcnow() - stats[["min_pts", "max_pts"]]

In [ ]:
stats[(stats.pts_missing_count>0) | 
      (stats.pts_decreasing_count>0) | 
      (stats.corrupted_count>0) | 
      (stats.discontinuity_count>100) | 
      (stats.idle_count>5)
     ].T

In [ ]:
df[df.description.isin(["statistics"])].groupby(["stream", "probe_name"]).last().T

In [ ]:
df.groupby(["stream", "description"]).size()

In [ ]:
df[df.level.isin(["WARN","DEBUG"])].groupby(["stream", "description", "probe_name"]).size()

In [ ]:
df[df.description.isin(["Gap in PTS is too large"])][["time_delta", "probe_name"]].value_counts().sort_index()

In [ ]:
df[df.description.isin(["PTS is missing"])][["pts", "time_delta", "probe_name"]]

In [ ]:
df[df.description.isin(["PTS is decreasing"])][["pts", "time_delta", "probe_name"]]

In [ ]:
df.iloc[300:400][df.probe_name.isin(["1-pravegasrc"])][["pts", "ms_from_prev", "prev_pts", "time_delta", "probe_name", "flags", "size", "description"]]

In [ ]:
df2 = df[df.description.isin(["buffer"]) & df.probe_name.isin(["1-pravegasrc"])].copy()

In [ ]:
df2["ms_from_prev"] = df2.pts.diff() / pd.Timedelta(1, "millisecond")

In [ ]:
df2[["pts", "ms_from_prev", "element", "pad", "flags", "size", "description"]].head(200)

In [ ]:
df2.ms_from_prev.value_counts()

In [ ]:
df2 = df[df.description.isin(["buffer"]) & ((df.element=="pravegasrc") | ((df.element=="h264parse") & (df["pad"]=="sink")))].copy()

In [ ]:
df2["ms_from_prev"] = df2.pts.diff() / pd.Timedelta(1, "millisecond")

In [ ]:
df2[["pts", "ms_from_prev", "element", "pad", "flags", "size", "description"]].head(200)

In [ ]:
df2[df2.element=="h264parse"].ms_from_prev.value_counts()

In [ ]:
df2.ms_from_prev.value_counts()